In [1]:
import os
import re

In [2]:
markupText_pattern = re.compile(r'<markupText>(.*?)</markupText>')
clean_pattern = re.compile('<.*?>')                                

In [3]:
dir_names = os.listdir('/data/atom/jp/data')

In [9]:
train_file = open('/data/scarlett/Japan/train.txt','w')
for dir_name in dir_names:
    dir_path = '/data/atom/jp/data/' + dir_name
    file_names = os.listdir(dir_path)
    for file_name in file_names:
        file_path = '/data/atom/jp/data/' + dir_name + '/' + file_name
        data = ' '.join(open(file_path).readlines())
        markupText = re.findall(markupText_pattern, data)
        markupText_ = []
        for item in markupText:
            item_clean = re.sub(clean_pattern,'',item)
            markupText_.append(item_clean)
        train_file.write(' '.join(markupText_)+'\n')

In [2]:
import sys
import json
import pymysql
import pandas as pd
import collections
from sqlalchemy import create_engine
conf = {
    'user': 'root',
    'password': 'rootpwd',
    'host': '10.123.4.200',
    'port': 3308,
    'db': 'muffin_dev',
    'charset': 'utf8',
}
conn = pymysql.connect(**conf)
engine = create_engine('mysql+pymysql://', creator=lambda:conn)

In [3]:
sql_info = '''
select first_table.headline, second_table.provision, second_table.caption, second_table.markup_text from
(SELECT document_id, headline FROM muffin_dev.japan_case_info where version = 6)first_table
join
(SELECT document_id, provision, caption, markup_text FROM muffin_dev.japan_law where version = 6)second_table
on
first_table.document_id = second_table.document_id
'''
df_info = pd.read_sql(sql_info,con=engine)

In [4]:
df_info = df_info.fillna('')

In [5]:
df_info.shape

(324572, 4)

In [6]:
df_info.head(2)

,headline,provision,caption,markup_text
0,消費者庁の所管する法律の規定に基づく立入検査等の際に携帯する職員の身分を示す証明書の様式の特...,,,次の各号に掲げる法律の規定（都道府県知事又は市町村長特別区の区長を含む。の事務に係るものに...
1,占領地軍政官憲ノ為シタル行為ノ法律上ノ効力等ニ関スル法律ノ施行ニ関スル件,,,１ 戸籍、船舶又ハ船員ニ関スル事項ニ付占領地軍政官憲ノ為シタル行為ニシテ戸籍法、民法、船舶法...


In [16]:
file_train_all = open('/data/scarlett/Japan/data/train_all_test.txt','w')

In [17]:
for item in df_info.iterrows():
    file_train_all.write(' '.join([item[1].headline, item[1].caption, item[1].markup_text]) + '\n')

In [9]:
import os
prefix = '/home/aidog/workspace/scarlett/Japan/pairwise/data'
label_laws = pd.read_excel(os.path.join(prefix,'excel_label_data.xlsx'),dtype='str')

In [10]:
label_laws = label_laws[['documenttitle','ChapterTitle','ChapterBody']]

In [11]:
label_laws = label_laws.fillna('')

In [12]:
label_laws.drop_duplicates().shape

(9225, 3)

In [13]:
label_laws.head(1)

,documenttitle,ChapterTitle,ChapterBody
0,(国内)補給部品・用品販売業務規程,,この規程は、年月日から実施する。


In [14]:
for item in label_laws.iterrows():
    file_train_all.write(' '.join([item[1].documenttitle, item[1].ChapterTitle, item[1].ChapterBody]) + '\n')

In [15]:
file_train_all.close()